Q1-1) 주석을 기반으로 코딩을 해주세요

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# parameters
learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [3]:
## MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [5]:
linear1 = torch.nn.Linear(784,100,bias=True)
linear2 = torch.nn.Linear(100,100,bias=True)
linear3 = torch.nn.Linear(100,10,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)

#nn_linear1 = torch.nn.Linear(784,100,bias=True)
#nn_linear2 = torch.nn.Linear(100,100,bias=True)
#nn_linear3 = torch.nn.Linear(100,10,bias=True)


In [6]:
## xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 1.8834e-01,  8.0911e-02, -5.7841e-02,  4.7004e-02,  5.4597e-02,
         -2.2657e-01,  7.0984e-02,  2.0391e-01,  2.6836e-02, -4.0637e-02,
         -2.0239e-01, -1.6156e-01,  2.0833e-01,  1.3363e-01, -3.3570e-02,
          1.0181e-01,  2.1810e-01, -1.0430e-01, -2.3068e-01, -8.5909e-02,
         -8.4788e-02,  1.2768e-02,  2.1248e-01,  1.1822e-01,  2.0103e-01,
         -1.4809e-01, -8.3738e-02,  1.0027e-01,  1.5828e-01, -1.0767e-01,
          1.9710e-01,  1.7866e-01, -2.0903e-01, -2.7886e-02, -1.8087e-01,
         -2.6533e-03, -9.0101e-02, -2.3290e-01, -1.7138e-01, -9.2445e-02,
          2.3105e-01,  1.0634e-01,  1.6106e-01,  9.7877e-02, -7.2314e-02,
         -2.1307e-01, -2.0362e-02,  6.5418e-02, -1.2234e-01,  1.1297e-01,
         -1.0994e-01,  3.3959e-02, -7.6807e-02,  1.1816e-01, -1.2633e-01,
          2.9554e-02,  9.9724e-02, -1.2055e-01, -2.7872e-02,  2.5077e-02,
         -8.4208e-02, -6.3669e-02, -1.5805e-02, -1.4308e-01, -1.9056e-02,
          5.1988

In [7]:
bn_model = torch.nn.Sequential(linear1,bn1,relu,dropout,
                               linear2,bn2,relu,dropout,
                               linear3).to(device)

#nn_model = torch.nn.Sequential(nn_linear1,relu,
#                              nn_linear2,relu,
#                              nn_linear3).to(device)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
bn_optimizer = torch.optim.Adam(bn_model.parameters(),lr=learning_rate)
#nn_optimizer = torch.optim.Adam(nn_model.parameters(),lr=learning_rate)

In [9]:
train_total_batch = len(train_loader)
print(train_total_batch)
test_total_batch = len(test_loader)
print(test_total_batch)

600
100


In [10]:
# training epoch
for epoch in range(training_epochs):
    avg_cost= 0
    bn_model.train()
    
    for x, y in train_loader:
        x = x.view(-1,784).to(device)
        y = y.to(device)
        
        bn_optimizer.zero_grad()
        bn_prediction = bn_model(x)
        bn_loss = criterion(bn_prediction,y)
        bn_loss.backward()
        bn_optimizer.step()
        
        avg_cost += bn_loss / train_total_batch 
        
    print('Epoch:', '%04d'% (epoch+1),'cost=','{:.9f}'.format(
        avg_cost))

print('Learning finished')

    
        

Epoch: 0001 cost= 0.493593425
Epoch: 0002 cost= 0.360244006
Epoch: 0003 cost= 0.324935347
Epoch: 0004 cost= 0.315062344
Epoch: 0005 cost= 0.303531557
Epoch: 0006 cost= 0.301063657
Epoch: 0007 cost= 0.285570860
Epoch: 0008 cost= 0.286456972
Epoch: 0009 cost= 0.262443125
Epoch: 0010 cost= 0.269874275
Epoch: 0011 cost= 0.261900514
Epoch: 0012 cost= 0.251416653
Epoch: 0013 cost= 0.247882366
Epoch: 0014 cost= 0.244977385
Epoch: 0015 cost= 0.245383218
Learning finished


In [11]:
with torch.no_grad():
    bn_model.eval()
    
    bn_acc = 0
    
    x_test = mnist_test.test_data.view(-1,784).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    for x, y in test_loader:
        
        x = x.view(-1,784).to(device)
        y = y.to(device)
        bn_prediction = bn_model(x)
        bn_correct = torch.argmax(bn_prediction,dim=1)== y
        bn_loss += criterion(bn_prediction,y)
        bn_acc += bn_correct.float().mean()
        
        
    bn_acc = bn_acc / test_total_batch
    
    print('Accuracy:',bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

C:\Users\lg\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\lg\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9665998816490173
Label:  5
Prediction:  6


Q1-2) layer들의 hiddne node 수를 증가/감소 시켰을 때 train set에서의 cost와 test set에서 accuracy가 어떻게 달라졌는지 비교해주세요

In [12]:
# parameters
learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [13]:
linear1 = torch.nn.Linear(784,300,bias=True)
linear2 = torch.nn.Linear(300,200,bias=True)
linear3 = torch.nn.Linear(200,10,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

bn1 = torch.nn.BatchNorm1d(300)
bn2 = torch.nn.BatchNorm1d(200)

In [14]:
## xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.0398,  0.1154,  0.1146,  ..., -0.1328,  0.0045, -0.1422],
        [ 0.0731,  0.0151,  0.0076,  ...,  0.1081,  0.1161,  0.1049],
        [ 0.0097, -0.1443, -0.0501,  ...,  0.0308, -0.0815, -0.1342],
        ...,
        [ 0.0095, -0.1313, -0.0983,  ..., -0.1069, -0.0204, -0.0300],
        [ 0.1610, -0.0986, -0.0824,  ..., -0.0932,  0.0568, -0.0663],
        [ 0.1536,  0.0194,  0.1441,  ...,  0.0955, -0.0695, -0.0033]],
       requires_grad=True)

In [15]:
new_model = torch.nn.Sequential(linear1,bn1,relu,dropout,
                               linear2,bn2,relu,dropout,
                               linear3).to(device)

In [16]:
criterion = torch.nn.CrossEntropyLoss().to(device)
bn_optimizer = torch.optim.Adam(new_model.parameters(),lr=learning_rate)

In [17]:
train_total_batch = len(train_loader)
print(train_total_batch)
test_total_batch = len(test_loader)
print(test_total_batch)

600
100


In [18]:
# training epoch
for epoch in range(training_epochs):
    avg_cost= 0
    new_model.train()
    
    for x, y in train_loader:
        x = x.view(-1,784).to(device)
        y = y.to(device)
        
        bn_optimizer.zero_grad()
        bn_prediction = new_model(x)
        bn_loss = criterion(bn_prediction,y)
        bn_loss.backward()
        bn_optimizer.step()
        
        avg_cost += bn_loss / train_total_batch 
        
    print('Epoch:', '%04d'% (epoch+1),'cost=','{:.9f}'.format(
        avg_cost))

Epoch: 0001 cost= 0.446044892
Epoch: 0002 cost= 0.318162203
Epoch: 0003 cost= 0.278368801
Epoch: 0004 cost= 0.249427304
Epoch: 0005 cost= 0.249380350
Epoch: 0006 cost= 0.232698008
Epoch: 0007 cost= 0.224990413
Epoch: 0008 cost= 0.212452903
Epoch: 0009 cost= 0.207218498
Epoch: 0010 cost= 0.192861184
Epoch: 0011 cost= 0.190346345
Epoch: 0012 cost= 0.183635920
Epoch: 0013 cost= 0.190055922
Epoch: 0014 cost= 0.184086949
Epoch: 0015 cost= 0.177622020


In [19]:
with torch.no_grad():
    new_model.eval()
    
    bn_acc = 0
    
    x_test = mnist_test.test_data.view(-1,784).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    for x, y in test_loader:
        
        x = x.view(-1,784).to(device)
        y = y.to(device)
        bn_prediction = new_model(x)
        bn_correct = torch.argmax(bn_prediction,dim=1)== y
        bn_loss += criterion(bn_prediction,y)
        bn_acc += bn_correct.float().mean()
        
        
    bn_acc = bn_acc / test_total_batch
    
    print('Accuracy:',bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = new_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9699001908302307
Label:  5
Prediction:  5


Epoch: 0015 cost  감소
Accuracy: 근소하게 증가